In [65]:
import yfinance as yf
import math

In [66]:
stock = yf.Ticker('TSLA')
stockInfo = stock.info

# EPS

In [67]:
def calc_eps(net_income_to_common, shares_outstanding):
    return net_income_to_common / shares_outstanding

calc_eps(stockInfo['netIncomeToCommon'], stockInfo['sharesOutstanding'])

3.038484681151332

# Volatility
Valid intervals: `[1m, 2m, 5m, 15m, 30m, 60m, 90m, 1h, 1d, 5d, 1wk, 1mo, 3mo]`

In [68]:
def calc_volatility(stock, period, interval):
    historic = stock.history(period=period, interval=interval)
    values = [x for x in historic['Close'] if not math.isnan(x)]
    print(historic['Close'], len(historic['Close']))

    mean = sum(values) / len(values)
    deviation = [price - mean for price in values]
    variance = sum([dev * dev for dev in deviation]) / len(values)
    std_dev = math.sqrt(variance)

    #Not completely sure if this is the exact formula that is needed/wanted?
    volatility = std_dev * math.sqrt(len(values))
    
    return volatility
    


calc_volatility(stock, '2mo', '1d')


Date
2022-08-25 00:00:00-04:00           NaN
2022-08-26 00:00:00-04:00    288.089996
2022-08-29 00:00:00-04:00    284.820007
2022-08-30 00:00:00-04:00    277.700012
2022-08-31 00:00:00-04:00    275.609985
2022-09-01 00:00:00-04:00    277.160004
2022-09-02 00:00:00-04:00    270.209991
2022-09-06 00:00:00-04:00    274.420013
2022-09-07 00:00:00-04:00    283.700012
2022-09-08 00:00:00-04:00    289.260010
2022-09-09 00:00:00-04:00    299.679993
2022-09-12 00:00:00-04:00    304.420013
2022-09-13 00:00:00-04:00    292.130005
2022-09-14 00:00:00-04:00    302.609985
2022-09-15 00:00:00-04:00    303.750000
2022-09-16 00:00:00-04:00    303.350006
2022-09-19 00:00:00-04:00    309.070007
2022-09-20 00:00:00-04:00    308.730011
2022-09-21 00:00:00-04:00    300.799988
2022-09-22 00:00:00-04:00    288.589996
2022-09-23 00:00:00-04:00    275.329987
2022-09-26 00:00:00-04:00    276.010010
2022-09-27 00:00:00-04:00    282.940002
2022-09-28 00:00:00-04:00    287.809998
2022-09-29 00:00:00-04:00    268.20

220.83243256493793